<a href="https://colab.research.google.com/github/manamendraJN/fashion-intelligence-platform/blob/feature%2Fbody-measurement-and-sizing/notebooks/body_measurement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cell 1 - Setup and Environment Configuration

In [1]:
# PURPOSE: Install required packages, import libraries, and configure the environment
# WHY: Sets up all dependencies needed for deep learning model training

import sys
from datetime import datetime, timezone

# Print session information for tracking
print(f"Student: manamendraJN | Date: {datetime.now(timezone. utc).strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Python:  {sys.version. split()[0]}")

# Install required packages silently (-q flag)
# timm: Pre-trained vision models library
# albumentations: Advanced image augmentation library
! pip install -q timm albumentations

# Core deep learning imports
import torch
import torch.nn as nn  # Neural network layers
import torch.optim as optim  # Optimizers (AdamW)
from torch.utils.data import Dataset, DataLoader  # Data loading utilities
import timm  # Pre-trained models (EfficientNet, ResNet, MobileNet)

# Data processing imports
import numpy as np  # Numerical operations
import pandas as pd  # CSV/dataframe handling
import cv2  # Image loading
from pathlib import Path  # File path operations
import json  # Save/load configuration files

# Image augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Utilities
from tqdm. auto import tqdm  # Progress bars
import matplotlib.pyplot as plt  # Plotting
import seaborn as sns  # Statistical visualizations
from sklearn.metrics import r2_score  # R² metric calculation

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Set device (GPU if available, else CPU)
# WHY: GPU is 10-100x faster for deep learning
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Print environment information
print(f"✅ PyTorch {torch.__version__} | Device: {device}")
if torch.cuda.is_available():
    print(f"✅ GPU:  {torch.cuda.get_device_name(0)} ({torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB)")
print("🎲 Running without fixed seeds (non-deterministic mode)")
# NOTE: No random seeds = different results each run (but more realistic generalization)

Student: manamendraJN | Date: 2025-12-11 04:53:48
Python:  3.12.12
✅ PyTorch 2.9.0+cu126 | Device: cuda
✅ GPU:  Tesla T4 (15.8 GB)
🎲 Running without fixed seeds (non-deterministic mode)
